In [42]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import os

class downloader():
    """
    xxxx
    Args:
        
    """
    def __init__(self, page):
        # Declaring None here is required for method setup_page.
        self._page = page
        self.setup_page()
        self.extract_pdf()
    
    
    @property
    def page(self):
        """
        Simply returns the page. 
        Keeping page as a property ensures when page is changed, 
        the correct pdf file is referred by the object (by calling self.setup_page).
        """
        return self._page
    
    
    @page.setter
    def page(self, value):
        """
        Setter for class property self.page.
        Keeping page as a property ensures when page is changed, 
        the correct pdf file is referred by the object (by calling self.setup_page).
        """
        self._page = value
        self.setup_page()
    
    
    @property
    def processed_page(self):
        """
        A class property. Convert the page number (self.page) into the page number of the pdf.
        Used in method self.extract_pdf
        """
        return self.page % 100
    
    
    def setup_page(self):
        # Construct the full pdf address
        pdf_folder_address='C:\\Users\\user1\\Documents\\GitHub\\JapaneseExtraction\\jp_book'
        pdf_start = 'jp_book_basic_'
        pdf_end = '.pdf'
        pdf_middle_lkup = [
            '001-100',
            '101-200',
            '201-300',
            '301-400',
            '401-500', 
            '501-600',
            '600-end'
        ]
        pdf_middle = pdf_middle_lkup[self.page//100]   
        pdf_name = pdf_start + pdf_middle + pdf_end
        pdf_full_address = os.path.join(pdf_folder_address, pdf_name)
        
        # Check if the current object's pdf address is the same. 
        # If same, no need to do anything to driver. Else close current and open new driver.
        try:
            if pdf_full_address == self._pdf_full_address:
                self._requires_driver_refresh = False
            else:
                self._pdf_full_address = pdf_full_address
                self._requires_driver_refresh = True
        except AttributeError:
            self._pdf_full_address = pdf_full_address
            self._requires_driver_refresh = True
        

    def extract_pdf(self):
        """
        Go through selenium process to get text from pdf. 
        Creates attribute driver, and subsequently also creates and fills attribute self.text
        
        Args:
            None
        Returns:
            None        
        """
        if self._requires_driver_refresh==True:
            if hasattr(self, 'driver'):
                if self.driver.session_id is not None:
                    self.driver.quit()                    
            
            self.driver = webdriver.Chrome()
            self.driver.get('https://www.newocr.com/')

            # Initial page, uplaod file
            pdf_selector = self.driver.find_element_by_name('userfile')
            pdf_selector.send_keys(self._pdf_full_address)

            self.driver.find_element_by_id('preview').click()

            # Preferences page, add Japanese language 
            self.driver.find_element_by_class_name('search-field').click()
            language_list = self.driver.find_elements_by_tag_name('li')
            language_list[58].click() # 58 is japanese
        
        # change page number of pdf and generate preview
        Select(self.driver.find_element_by_name('p')).select_by_value(str(self.processed_page))
        
        self.driver.find_element_by_name('ocr').click()
    
    
    # def get_text_file(self):
        ## The following code is for downloading the text file
        # driver.find_element_by_class_name('dropdown-toggle').click()
        # dl_dropdown = driver.find_element_by_class_name('dropdown-menu')
        # dl_dropdown.find_element_by_tag_name('li').click()
    
    
    
    


In [43]:
dl = downloader(207)

In [46]:
dl.page = 351

In [47]:
dl.extract_pdf()